In [48]:
import numpy as np
import gdal as gd
import scipy.ndimage
import matplotlib.pyplot as plt
import osr
from numpy import ma
from scipy import interpolate

#write filename of raster (must be .tif file) here; should be in same folder this program is in
file = "doris.tif"
crater = gd.Open(file)
prj = crater.GetProjection()
print(prj)

#this is the dimensions of the raster image in pixel coordinates
width = crater.RasterXSize
height = crater.RasterYSize
print(width)
print(height)

#the following commands parse out the projection of your raster image, giving you the top left and bottom right points
gt = crater.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 
print(minx)
print(miny)
print(maxx)
print(maxy)

#using the geographic coordinate ranges, scales to the pixel coordinate range
OldXRange = (maxx - minx)
OldYRange = (miny - maxy)
NewXRange = (height - 1)
NewYRange = (width - 1)

#here are the two center points of the crater in meters, taken from the projected .tif file
CraterCentX = 7687.5
CraterCentY = ((-1)*(maxy - (-1845337.5)))+miny

#insert the x and y values in geographic coordinates to convert point to pixel coordinates
NewXValue = (((CraterCentX - minx) * NewXRange) / OldXRange) + 1
NewYValue = (((CraterCentY - miny) * NewYRange) / OldYRange) + 1
print("center coordinates in pixels are")
print(NewXValue)
print(NewYValue)

#insert diameter of crater or central peak below, in meters
CraterDiam = 14500

#resolution of Herrick DEMs is 225 m/pixel
Radius = ((CraterDiam/2)/225)
print("radius is (in pixels)")
print(Radius)

#print(gdaltransform(crater))

#this allows us to get turn the raster into a NumPy array
crater_array = np.array(crater.GetRasterBand(1).ReadAsArray())
#print("this is the crater array")
#print(crater_array)

# construct interpolation function    
x = np.arange(crater_array.shape[1])
y = np.arange(crater_array.shape[0])
f = scipy.interpolate.interp2d(x, y, crater_array, kind='linear')

def PlotCircle():
    thetas = np.arange(0, 360, 1)
    r = Radius
    x0, y0 = 100, 76
    xn1, yn1 = x0, (y0+r)
    add = (r + y0) - yn1
    results = []
    for theta in thetas:
        xn = xn1 + r*(np.sin(np.deg2rad(theta)))
        yn = yn1 - r*(1 - np.cos(np.deg2rad(theta)))+add
        num_points = 100
        xvalues = np.linspace(x0, xn, num_points)
        yvalues = np.linspace(y0, yn, num_points)
        zvalues = f(xvalues, yvalues)
        zi = scipy.ndimage.map_coordinates(np.transpose(crater_array), np.vstack((xvalues,yvalues)), order=1)
        crater_depths = np.max(zi)-np.min(zi)
        fig, axes = plt.subplots(nrows=2)
        axes[0].imshow(crater_array)
        axes[0].plot([x0, xn], [y0, yn], 'ro-')
        axes[1].plot(zi)
        plt.ylabel("Depth (m)")
        plt.xlabel("Transect (points)")
        if xn > 0:
            results.append(crater_depths)
        else:
            print("value is not real")
    print(zi)
    print("these are the crater depths")
    print(results)
    #print(len(results))
    #results[:] = [value for value in results if value >= np.average(results)]
    crater_depths_sum = 0
    #print(results[:])
    #print(len(results[:]))
    for i in results:
        crater_depths_sum = i + crater_depths_sum
    print(crater_depths_sum)
    crater_depths_avg = crater_depths_sum/len(results)
    print("this is the average crater depth")
    print(crater_depths_avg)
    cXs = []
    cYs = []
    for t in thetas:
        cXs.append(x0 + r*np.cos(np.deg2rad(t)))
        cYs.append(y0 + (r*(1 - np.sin(np.deg2rad(t)))))
    #plt.plot(cXs, cYs, 'g-')
    #plt.show()
    #plt.clf()
    xn1 = xn
    yn1 = yn
PlotCircle()

PROJCS["ven_sin_doris",GEOGCS["GCS_Venus_1985",DATUM["D_Venus_1985",SPHEROID["Venus_1985_IAU_IAG_COSPAR",6051000,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",90],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]
176
164
263287.5
219787.5
302887.5
256687.5
center coordinates in pixels are
-1051.090909090909
9969.953252032521
radius is (in pixels)
32.22222222222222


/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


[6473 6469 6465 6461 6458 6454 6450 6446 6442 6438 6435 6431 6427 6425 6423
 6422 6421 6419 6418 6416 6415 6414 6412 6411 6409 6411 6417 6423 6429 6436
 6442 6448 6454 6460 6466 6472 6478 6484 6488 6492 6495 6499 6503 6507 6510
 6514 6518 6522 6525 6529 6536 6543 6551 6559 6567 6575 6582 6590 6597 6605
 6613 6621 6635 6653 6672 6690 6709 6727 6746 6764 6783 6802 6820 6839 6856
 6867 6878 6890 6901 6912 6924 6935 6946 6958 6969 6981 6992 7000 7008 7015
 7023 7030 7037 7045 7052 7060 7067 7074 7082 7084]
these are the crater depths
[674, 676, 677, 679, 680, 681, 683, 684, 680, 676, 672, 666, 660, 652, 642, 632, 621, 610, 601, 593, 584, 576, 570, 577, 586, 595, 603, 611, 618, 625, 630, 627, 622, 618, 615, 609, 606, 602, 598, 591, 579, 566, 553, 540, 528, 520, 513, 508, 506, 504, 500, 495, 490, 487, 485, 484, 485, 485, 490, 493, 498, 498, 496, 495, 495, 495, 496, 497, 498, 496, 495, 494, 494, 493, 494, 495, 498, 501, 504, 508, 510, 512, 514, 516, 523, 530, 535, 540, 544, 550, 556, 560, 566